In [14]:
import harmonica as hm
import xarray as xr

In [15]:
path = '/Users/thowe/Desktop/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.grd'

da = hm.load_oasis_montaj_grid(fname=path)

In [16]:
# Print basic info
print("Dimensions:", da.dims)
print("Shape:", da.shape)
print("Coordinates:", list(da.coords))

# Print all attributes
print("\nAttributes:")
for attr_name, attr_value in da.attrs.items():
    print(f"  {attr_name}: {attr_value}")

# Print data summary
print("\nData Summary (ignoring NaNs):")
print("  Min:", float(da.min(skipna=True)))
print("  Max:", float(da.max(skipna=True)))
print("  Mean:", float(da.mean(skipna=True)))
print("  Std Dev:", float(da.std(skipna=True)))
print("  % NaNs:", float(da.isnull().sum() / da.size * 100), "%")

Dimensions: ('northing', 'easting')
Shape: (28253, 22760)
Coordinates: ['easting', 'northing']

Attributes:
  n_bytes_per_element: 4
  sign_flag: 2
  shape_e: 22760
  shape_v: 28253
  ordering: 1
  spacing_e: 75.0
  spacing_v: 75.0
  x_origin: -876975.0
  y_origin: 13500.0
  rotation: 0.0
  base_value: 0.0
  data_factor: 1.0
  map_projection: 0
  units_x: 0
  units_y: 0
  units_z: 0
  n_valid_points: 195867186
  grid_min: -219.20159912109375
  grid_max: 363.04364013671875
  grid_median: 0.0
  grid_mean: -0.0002776627952698618
  grid_variance: 0.9875565402201316
  process_flag: 0

Data Summary (ignoring NaNs):
  Min: -219.20159912109375
  Max: 363.04364013671875
  Mean: -0.00027766279840912817
  Std Dev: 0.9937587912455751
  % NaNs: 69.5403536473754 %


In [17]:
# Export to NetCDF file.
output_path = '/Users/thowe/Desktop/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.nc'

# Wrap in a Dataset with a variable name
wrapped = xr.Dataset({"magnetics": ds})

# Save it
wrapped.to_netcdf(output_path)